This is a starter notebook for the project, you'll have to import the libraries you'll need, you can find a list of the ones available in this workspace in the requirements.txt file in this workspace. 

In [1]:
import os

os.environ["OPENAI_API_KEY"] = "your-key-here"
os.environ["OPENAI_API_BASE"] = "https://openai.vocareum.com/v1"

from langchain.chat_models import ChatOpenAI

# Generate Real Estate Listings

In [2]:
query = """Generate real estate listing, including the following information:
    Neighborhood: name of neighborhood
    Price: house price in US dollar
    Bedrooms: number of bedrooms
    Bathrooms: number of bathrooms
    House Size: house area in sqft
    Description: description of the house
    Neighborhood Description: description of the neighborhood
"""
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=1.0)

In [3]:
real_estate_listings = []

for i in range(10):
    res = llm.predict(query).strip()
    real_estate_listings.append(res)

In [4]:
for i in range(10):
    print(f"### example number {i+1} ###")
    print(real_estate_listings[i])

### example number 1 ###
Neighborhood: Sunset Heights
Price: $500,000
Bedrooms: 3
Bathrooms: 2
House Size: 1,800 sqft
Description: Beautifully renovated home in the sought after Sunset Heights neighborhood. This charming 3 bedroom, 2 bathroom house features a spacious open floor plan, updated kitchen with stainless steel appliances, hardwood floors throughout, and a cozy backyard perfect for entertaining. Don't miss out on this gem!

Neighborhood Description: Sunset Heights is a vibrant and family-friendly neighborhood known for its tree-lined streets, local parks, and great schools. Conveniently located near top-rated restaurants, shops, and easy access to major highways for quick commute to downtown. This is the ideal place to call home for those looking for a mix of urban living and suburban charm.
### example number 2 ###
Neighborhood: Highlands Ranch, CO
Price: $550,000
Bedrooms: 4
Bathrooms: 3
House Size: 2,500 sqft
Description: This stunning 4 bedroom, 3 bathroom home in Highlan

In [5]:
with open("Listings.txt", "w") as f:
    for i in range(10):
        f.write(f"### example number {i+1} ###\n")
        f.write(real_estate_listings[i]+"\n")

In [6]:
from langchain.docstore.document import Document

docs = []
for i in range(len(real_estate_listings)):
    docs.append(Document(page_content=real_estate_listings[i], metadata={"row":i}))

print(docs)

[Document(page_content="Neighborhood: Sunset Heights\nPrice: $500,000\nBedrooms: 3\nBathrooms: 2\nHouse Size: 1,800 sqft\nDescription: Beautifully renovated home in the sought after Sunset Heights neighborhood. This charming 3 bedroom, 2 bathroom house features a spacious open floor plan, updated kitchen with stainless steel appliances, hardwood floors throughout, and a cozy backyard perfect for entertaining. Don't miss out on this gem!\n\nNeighborhood Description: Sunset Heights is a vibrant and family-friendly neighborhood known for its tree-lined streets, local parks, and great schools. Conveniently located near top-rated restaurants, shops, and easy access to major highways for quick commute to downtown. This is the ideal place to call home for those looking for a mix of urban living and suburban charm.", metadata={'row': 0}), Document(page_content="Neighborhood: Highlands Ranch, CO\nPrice: $550,000\nBedrooms: 4\nBathrooms: 3\nHouse Size: 2,500 sqft\nDescription: This stunning 4 be

# Storing Listings in a Vector Database

In [7]:
from langchain.text_splitter import CharacterTextSplitter

splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=1000,
    chunk_overlap=0,
)
splitted_docs = splitter.split_documents(docs)
print(splitted_docs)

[Document(page_content="Neighborhood: Sunset Heights\nPrice: $500,000\nBedrooms: 3\nBathrooms: 2\nHouse Size: 1,800 sqft\nDescription: Beautifully renovated home in the sought after Sunset Heights neighborhood. This charming 3 bedroom, 2 bathroom house features a spacious open floor plan, updated kitchen with stainless steel appliances, hardwood floors throughout, and a cozy backyard perfect for entertaining. Don't miss out on this gem!\nNeighborhood Description: Sunset Heights is a vibrant and family-friendly neighborhood known for its tree-lined streets, local parks, and great schools. Conveniently located near top-rated restaurants, shops, and easy access to major highways for quick commute to downtown. This is the ideal place to call home for those looking for a mix of urban living and suburban charm.", metadata={'row': 0}), Document(page_content="Neighborhood: Highlands Ranch, CO\nPrice: $550,000\nBedrooms: 4\nBathrooms: 3\nHouse Size: 2,500 sqft\nDescription: This stunning 4 bedr

In [8]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma

embeddings = OpenAIEmbeddings()
db = Chroma.from_documents(splitted_docs, embeddings)

# Building the User Preference Interface

In [9]:
questions = [   
                "How big do you want your house to be?",
                "What are 3 most important things for you in choosing this property?", 
                "Which amenities would you like?", 
                "Which transportation options are important to you?",
                "How urban do you want your neighborhood to be?",   
            ]
answers = [
    "A comfortable three-bedroom house with a spacious kitchen and a cozy living room.",
    "A quiet neighborhood, good local schools, and convenient shopping options.",
    "A backyard for gardening, a two-car garage, and a modern, energy-efficient heating system.",
    "Easy access to a reliable bus line, proximity to a major highway, and bike-friendly roads.",
    "A balance between suburban tranquility and access to urban amenities like restaurants and theaters."
]

In [10]:
from langchain.memory import ChatMessageHistory

history = ChatMessageHistory()
for i in range(len(questions)):
    history.add_ai_message(questions[i])
    history.add_user_message(answers[i])
print(history.messages)

[AIMessage(content='How big do you want your house to be?'), HumanMessage(content='A comfortable three-bedroom house with a spacious kitchen and a cozy living room.'), AIMessage(content='What are 3 most important things for you in choosing this property?'), HumanMessage(content='A quiet neighborhood, good local schools, and convenient shopping options.'), AIMessage(content='Which amenities would you like?'), HumanMessage(content='A backyard for gardening, a two-car garage, and a modern, energy-efficient heating system.'), AIMessage(content='Which transportation options are important to you?'), HumanMessage(content='Easy access to a reliable bus line, proximity to a major highway, and bike-friendly roads.'), AIMessage(content='How urban do you want your neighborhood to be?'), HumanMessage(content='A balance between suburban tranquility and access to urban amenities like restaurants and theaters.')]


In [11]:
# this memory only store summary of chat history
from langchain.memory import ConversationSummaryMemory

summary_memory = ConversationSummaryMemory.from_messages(
    llm=ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0),
    chat_memory=history,
    memory_key="summary",
    return_messages=True
)
print(summary_memory.buffer)

The AI asks the human how big they want their house to be, and the human responds that they want a comfortable three-bedroom house with a spacious kitchen and a cozy living room. The AI then asks the human about the three most important things in choosing the property, to which the human replies a quiet neighborhood, good local schools, and convenient shopping options. The AI then inquires about the amenities the human would like, and the human mentions a backyard for gardening, a two-car garage, and a modern, energy-efficient heating system. When asked about transportation options, the human mentions easy access to a reliable bus line, proximity to a major highway, and bike-friendly roads. The AI then asks how urban the human wants their neighborhood to be, and the human desires a balance between suburban tranquility and access to urban amenities like restaurants and theaters.


In [12]:
summary_memory.load_memory_variables({})

{'summary': [SystemMessage(content='The AI asks the human how big they want their house to be, and the human responds that they want a comfortable three-bedroom house with a spacious kitchen and a cozy living room. The AI then asks the human about the three most important things in choosing the property, to which the human replies a quiet neighborhood, good local schools, and convenient shopping options. The AI then inquires about the amenities the human would like, and the human mentions a backyard for gardening, a two-car garage, and a modern, energy-efficient heating system. When asked about transportation options, the human mentions easy access to a reliable bus line, proximity to a major highway, and bike-friendly roads. The AI then asks how urban the human wants their neighborhood to be, and the human desires a balance between suburban tranquility and access to urban amenities like restaurants and theaters.')]}

# Searching Based on Preferences

Retrieval logics:

- Iterate to each preference, search top 3 listings and add 1 matching score to these 3 listings

- Sort and extract top 3 listings with highest score

In [13]:
score_list = [0 for _ in range(10)]

for answer in answers:
    similar_listings = db.similarity_search(answer, k=3)
    for i in range(3):
        index = similar_listings[i].metadata["row"]
        score_list[index] += 1

print(score_list)

[1, 1, 0, 4, 4, 0, 0, 0, 0, 5]


In [14]:
sorted_indices = sorted(range(len(score_list)), key=lambda i: score_list[i], reverse=True)[:3]
print(sorted_indices)

[9, 3, 4]


In [19]:
for i in range(len(sorted_indices)):
    print(f"### {i+1}-th most matching listing ###")
    print(docs[sorted_indices[i]].page_content)

### 1-th most matching listing ###
Neighborhood: The Grove
Price: $500,000
Bedrooms: 3
Bathrooms: 2.5
House Size: 2,000 sqft
Description: This charming 3 bedroom, 2.5 bathroom home in The Grove neighborhood is the perfect mix of modern updates and original charm. The open concept living area features hardwood floors, a cozy fireplace, and plenty of natural light. The kitchen boasts stainless steel appliances, granite countertops, and a breakfast bar. The master suite includes a walk-in closet and ensuite bathroom with dual sinks and a soaking tub. Outside, the fenced backyard offers a deck for entertaining and a storage shed. Don't miss out on this gem!

Neighborhood Description: The Grove is a family-friendly neighborhood known for its tree-lined streets, parks, and top-rated schools. Residents enjoy easy access to shopping, dining, and entertainment options, as well as quick commutes to downtown and major highways. With a strong sense of community and plenty of amenities, The Grove i

# Personalizing Listing Descriptions

In [16]:
query_init = """You are given a house's information along with a chat summary between a bot and a buyer. Your job is to re-write the house description which emphasizes aspects of the property that align with what the buyer is looking for. Use correct information from the house information. Do not make up any information"""

In [17]:
from langchain.prompts import PromptTemplate

prompt = PromptTemplate(
    template="{query_init}\nHouse information:\n{house_info}\nChat summary:\n{chat_summary}",
    input_variables=["query_init", "house_info", "chat_summary"]
)
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

In [18]:
for i in range(len(sorted_indices)):
    print(f"###### {i+1}-th CANDIDATE ######")
    query = prompt.format(query_init=query_init, house_info=splitted_docs[sorted_indices[i]].page_content, chat_summary=summary_memory.buffer)
    print("=== query ===")
    print(query)
    output = llm.predict(query).strip()
    print("=== output ===")
    print(output)

###### 1-th CANDIDATE ######
=== query ===
You are given a house's information along with a chat summary between a bot and a buyer. Your job is to re-write the house description which emphasizes aspects of the property that align with what the buyer is looking for. Do not make up any information
House information:
Neighborhood: Beverly Hills
Price: $5,000,000
Bedrooms: 5
Bathrooms: 6
House Size: 5,000 sqft
Description: Stunning luxury home in the heart of Beverly Hills. This modern masterpiece features high-end finishes, an open floor plan, and floor-to-ceiling windows offering panoramic views of the city. The gourmet kitchen is a chef's dream with top-of-the-line appliances and a spacious island. The outdoor space is perfect for entertaining with a pool, spa, and outdoor kitchen. The master suite boasts a fireplace, dual walk-in closets, and a spa-like ensuite bathroom. Additional features include a home theater, wine cellar, and automated smart home system.
Neighborhood Description: 